In [20]:
import os
# os.getcwd()
os.chdir("/localdisk4/panwla/Projects/park_vlm/Kangaroo")
os.getcwd()

'/localdisk4/panwla/Projects/park_vlm/Kangaroo'

In [21]:
import json
import os
import boto3
import yaml
# from google.colab import drive

# # Mount Google Drive to access API keys
# drive.mount('/content/drive', force_remount=True)

# Load API keys from file
file_path = './.API_KEYS/API_KEYS.yml'

with open(file_path, 'r') as file:
    api_keys = yaml.safe_load(file)

### WANDB Keys
wandb_key = api_keys['WANDB']['Key']
hf_read_api_key = api_keys['HUGGINGFACE']['HF_READ_API_KEY']

# Extract AWS credentials
aws_access_key_id = api_keys['AWS']['AWS_ACCESS_KEY_ID']
aws_secret_access_key = api_keys['AWS']['AWS_SECRET_ACCESS_KEY']

from huggingface_hub import login
login(hf_read_api_key)

# Initialize the S3 client with credentials
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

In [22]:
import os

# Set environment variables based on your bashrc settings
os.environ["JUPYTER_DATA_DIR"] = "/localdisk4/panwla/jupyter_data"
os.environ["HF_HOME"] = "/localdisk4/panwla/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "/localdisk4/panwla/huggingface_cache/transformers"
os.environ["HF_DATASETS_CACHE"] = "/localdisk4/panwla/huggingface_cache/datasets"
os.environ["TORCH_HOME"] = "/localdisk4/panwla/torch"
os.environ["XDG_CACHE_HOME"] = "/localdisk4/panwla_cache/.cache"
os.environ["TMPDIR"] = "/localdisk4/panwla/tmp"
os.environ["TEMP"] = "/localdisk4/panwla/tmp"
os.environ["TMP"] = "/localdisk4/panwla/tmp"

# Create directories if they don't exist
directories = [
    os.environ["JUPYTER_DATA_DIR"],
    os.environ["HF_HOME"],
    os.environ["TRANSFORMERS_CACHE"],
    os.environ["HF_DATASETS_CACHE"],
    os.environ["TORCH_HOME"],
    os.environ["XDG_CACHE_HOME"],
    os.environ["TMPDIR"],
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)


In [23]:
# Download the model checkpoint to a local directory
import os
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Ensure HF_HOME & TMPDIR are set (if needed)
os.environ["HF_HOME"] = "/localdisk4/panwla/huggingface_cache"
os.environ["TMPDIR"] = "/localdisk4/panwla/tmp"

checkpoint_path = snapshot_download(
    repo_id="KangarooGroup/kangaroo",
    cache_dir="/localdisk4/panwla/huggingface_cache",
    resume_download=True,
    max_workers=1,  # reduce concurrency to avoid timeouts
)

print("Model checkpoint downloaded to:", checkpoint_path)

tokenizer = AutoTokenizer.from_pretrained("KangarooGroup/kangaroo")

print("Model will be imported now")

# Option 1: Use a specific device (this worked before)
device = torch.device("cuda:0")
model = AutoModelForCausalLM.from_pretrained(
    "KangarooGroup/kangaroo",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
model = model.to(device)

print("Model shifted to cuda successfully")

# Print device mapping information
for name, param in model.named_parameters():
    print(f"{name} is on {param.device}")

# Define terminators
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]


In [24]:
def analyze_video(video_path, query):
    
    # Check if video file exists
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video file not found: {video_path}")
    
    # Define terminators (constant across all calls)
    terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    
    # Run the model's chat function with fixed parameters
    output, _ = model.chat(
        video_path=video_path,
        query=query,
        tokenizer=tokenizer,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    return output

In [25]:

facial_expressions_query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''


In [26]:
Apparent_diff_task ='''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video. 

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: “Yes” or “No”.

Answer: 

'''


BG_and_lighting_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Mention if the background is overloaded (i.e., too many things), or the lighting condition is inappropriate (i.e., too dark or overlit). Otherwise, just output “normal background”.

Answer: 

'''


Blink_rate_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Was there anything abnormal about the person’s eye blink rate? For example, they may not be blinking at all, or they may have reduced or increased blink rate compared to a normal person. If there is nothing abnormal, output “normal blinking”.

Answer: 

'''


Camera_position_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: How far is the camera? A good position of the camera would be when the upper half of the subject's body remains visible, while the lower half is not captured in the frame. If the upper body is only partly visible, the camera is too close. If the lower body is also visible, the camera is too far.

Answer: 

'''


Coherence_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>

Question: Is the subject coherent in what they are speaking? Are they delivering an easy to understand story, or are they deterring a lot from the central topic and delivering an unorganized speech?

Answer: 

'''


Comp_task_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>

Question: Please indicate whether the subject was able to follow the instructions while completing the task. If the subject was doing something differently, please describe.

Answer: 


'''

Lip_parting_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate the extent of lips parting when the subject is not saying anything (i.e., always/most of the times/sometimes/very few times/never).

Answer: 

'''


Masked_faceExp_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate which of the following are true for the subject: (i) The individual's face appears blank and emotionless, even when they are trying to express an emotion. (ii) The expression is weak or asymmetrical, and the individual has difficulty holding an expression (e.g., smile) for an extended period. Also mention if you observe other facial expression abnormalities.

Answer: 

'''

Observations_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Document any abnormal signs observed in body parts other than the face. This includes, but is not limited to, tremors in the hands, involuntary shaking or rhythmic movements of the upper or lower extremities, stiffness or rigidity in the limbs, reduced arm swing while speaking, or any signs of bradykinesia (slowness of movement). Additionally, note any abnormal postures, difficulty in maintaining balance, or other motor irregularities that may be indicative of Parkinson’s disease.

Answer: 


'''

Overall_app_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Provide a brief description of the subject's perceived state of mind, noting whether they appear energetic, exhausted, calm, confused, or exhibit any other relevant emotional or cognitive cues.

Answer: 


'''


Other_people_present_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate whether any other individuals were present in the background. If so, provide a brief description (e.g., "An older male is visible in the background"). Conclude with a final answer: "Yes" or "No".

Answer: 
 

'''


Comp_sent_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>


Question: Indicate whether the subject is using complex sentences that are difficult to understand. Conclude with a final answer: "Yes" or "No".

Answer: 


'''


Other_BP_visible_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate if any body part critical for this task is partially visible (or invisible). For example, was the subject wearing a mask that may have obstructed important visual information? Or did the face go out of frame while the subject was completing the task?

Answer: 


'''

In [27]:
# === Model inference function (as provided) ===
def analyze_video(video_path, query):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video file not found: {video_path}")

    terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

    output, _ = model.chat(
        video_path=video_path,
        query=query,
        tokenizer=tokenizer,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return output

In [28]:
import os
import pandas as pd

# === Setup file paths ===
cayla_files = "/localdisk4/panwla/Projects/park_vlm/Annotations/Clinical/Cayla/cayla_common_final.csv"
nami_files = "/localdisk4/panwla/Projects/park_vlm/Annotations/Clinical/Nami/nami_common_final.csv"
natalia_files = "/localdisk4/panwla/Projects/park_vlm/Annotations/Clinical/Natalia/natalia_common_final.csv"

video_path = "/localdisk4/panwla/Projects/park_vlm/Videos/Videos"

# === Load video references ===
cayla_df = pd.read_csv(cayla_files)
nami_df = pd.read_csv(nami_files)
natalia_df = pd.read_csv(natalia_files)

# List of column indices to select
columns_to_keep = [0, 1, 2, 3, 5, 6, 8, 14, 18, 21, 24, 28, 29, -1]

# Subset Cayla
cayla_df = cayla_df.iloc[:, columns_to_keep]

# Identify prompt + video columns
prompt_columns = list(cayla_df.columns[0:13])  # first 13 columns
video_column = ['video']

# Subset Nami and Natalia using same columns
nami_df = nami_df.loc[:, prompt_columns + video_column]
natalia_df = natalia_df.loc[:, prompt_columns + video_column]

# List available videos
available_videos = os.listdir(video_path)

# === Empty DataFrame to collect results ===
columns = ["Video_file", "Q_ID", "Prompt_name", "Model_Name", "Cayla_Resp", "Nami_Resp", "Natalia_Resp"]
output = pd.DataFrame(columns=columns)


In [29]:

# === Store prompts in a dictionary ===
prompt_dict = {
    "facial_expressions_query": facial_expressions_query,
    "BG_and_lighting_query": BG_and_lighting_query,
    "Blink_rate_query": Blink_rate_query,
    "Camera_position_query": Camera_position_query,
    "Coherence_query": Coherence_query,
    "Comp_task_query": Comp_task_query,
    "Lip_parting_query": Lip_parting_query,
    "Masked_faceExp_query": Masked_faceExp_query,
    "Observations_query": Observations_query,
    "Overall_app_query": Overall_app_query,
    "Other_people_present_query": Other_people_present_query,
    "Other_BP_visible_query": Other_BP_visible_query,
    
}



In [30]:
cayla_df.head(2)

In [31]:
cayla_df.head(1)

In [32]:
natalia_df.head(1)

In [33]:
nami_df.head(1)

In [34]:
cayla_df.shape, nami_df.shape, natalia_df.shape

In [ ]:
nami_df.columns

In [36]:
 

set(prompt_columns)- set(nami_df[prompt_columns].columns)

In [37]:
set(prompt_columns)- set(cayla_df.columns), set(prompt_columns)- set(nami_df.columns), set(prompt_columns)- set(natalia_df.columns)




In [38]:
cayla_df['video'][0]

In [39]:
############### Load Whishper Model #########
import torch
from transformers import (
    AutoModelForSpeechSeq2Seq,
    AutoProcessor,
    pipeline
)

# Optional: select your device
device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "openai/whisper-large-v3"
torch_dtype = torch.float16 if "cuda" in device else torch.float32

print(f"Loading model '{model_id}' on device '{device}', dtype={torch_dtype}")
whishper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
whishper_model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# Create the pipeline just once
pipe_asr = pipeline(
    "automatic-speech-recognition",
    model=whishper_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=0 if device.startswith("cuda") else -1
)

In [40]:
import os
import pandas as pd
from pydub import AudioSegment
import tempfile

def transcribe_videos(input_df, video_path, pipe_asr):
    input_df = input_df.copy()
    if "Transcription" not in input_df.columns:
        input_df["Transcription"] = None
    
    available_videos = [f for f in os.listdir(video_path) if f.endswith(".mp4")]
    print(f"Found {len(available_videos)} videos in directory")
    
    for idx, row in input_df.iterrows():
        video_name = row.get("video", None)
        if not isinstance(video_name, str) or not video_name:
            continue
        
        # Find exact matching file
        matched_file = None
        for filename in available_videos:
            if video_name[-25:] in filename:
                matched_file = filename
                break
        
        if matched_file:
            full_video_path = os.path.join(video_path, matched_file)
            print(f"\n🎬 Found match for row {idx}: {matched_file}")
            print(f"Full path: {full_video_path}")
            
            try:
                # Use a temporary file with unique name
                with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as temp_audio:
                    tmp_audio_file = temp_audio.name
                
                # Convert video to audio
                audio = AudioSegment.from_file(full_video_path, format="mp4")
                audio = audio.set_channels(1).set_frame_rate(16000)
                audio.export(tmp_audio_file, format="wav")
                
                # Perform transcription
                result = pipe_asr(tmp_audio_file)
                transcription_text = result["text"]
                
                # Add to dataframe
                input_df.loc[idx, "Transcription"] = transcription_text
                print(f"📝 Transcription: {transcription_text}")
                
                # Clean up temp file
                os.remove(tmp_audio_file)
                
            except Exception as e:
                print(f"⚠️ Failed to process video {matched_file}: {e}")
        else:
            print(f"❌ No matching video found for row {idx}, video name: {video_name}")
    
    return input_df

def process_all_prompts(input_df, prompt_dict, video_path, analyze_video, nami_df=None, natalia_df=None):
    available_videos = [f for f in os.listdir(video_path) if f.endswith(".mp4")]
    
    # List of prompts that need transcription
    transcription_prompts = ["Coherence_query", "Comp_task_query", "Comp_sent_query"]
    
    # Create output DataFrame
    columns = ["Video_file", "Q_ID", "Prompt_name", "Model_Name", "Output", "Cayla_Resp", "Nami_Resp", "Natalia_Resp"]
    output = pd.DataFrame(columns=columns)
    
    # Process each row in the input DataFrame
    for idx, row in input_df.iterrows():
        video_name = row.get("video", None)
        if not isinstance(video_name, str) or not video_name:
            continue
            
        # Get the transcription
        transcription = row.get("Transcription", "")
        
        # Find matching video file
        matched_file = None
        for f in available_videos:
            if video_name[-25:] in f:
                matched_file = f
                break
    
        if matched_file:
            full_video_path = os.path.join(video_path, matched_file)
            print(f"\n🎬 Processing video: {matched_file}")
            
            # Get prompt columns for annotations
            prompt_columns = list(input_df.columns[0:13])  # First 13 columns as per original code
            
            # Process each prompt in the dictionary
            for q_id, (prompt_name, prompt_text) in enumerate(prompt_dict.items(), start=1):
                # For the 3 specific prompts, insert transcription if available
                if prompt_name in transcription_prompts and isinstance(transcription, str) and transcription:
                    prompt_text = prompt_text.replace("<TRANSCRIPTION_OUTPUT>", transcription)
                    print(f"🔄 Modified prompt {prompt_name} with transcription")
                
                # Get responses for comparison
                cayla_resp = row[prompt_columns[q_id - 1]] if q_id - 1 < len(prompt_columns) else ""
                
                nami_resp = ""
                if nami_df is not None:
                    nami_row = nami_df[nami_df["video"] == video_name]
                    if not nami_row.empty and q_id - 1 < len(prompt_columns):
                        nami_resp = nami_row[prompt_columns[q_id - 1]].values[0]
                
                natalia_resp = ""
                if natalia_df is not None:
                    natalia_row = natalia_df[natalia_df["video"] == video_name]
                    if not natalia_row.empty and q_id - 1 < len(prompt_columns):
                        natalia_resp = natalia_row[prompt_columns[q_id - 1]].values[0]
                
                # Process with Kangaroo model
                response = analyze_video(full_video_path, prompt_text)
                print(f"✅ Completed prompt: {prompt_name}")
                
                # Add to results DataFrame
                new_row = {
                    "Video_file": matched_file,
                    "Q_ID": q_id,
                    "Prompt_name": prompt_name,
                    "Model_Name": "Kangaroo",
                    "Output": response,
                    "Cayla_Resp": cayla_resp,
                    "Nami_Resp": nami_resp,
                    "Natalia_Resp": natalia_resp
                }
                
                output = pd.concat([output, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"❌ No matching video found for: {video_name}")
    
    return output

# Example usage:

# First transcribe the videos
cayla_df_with_transcriptions = transcribe_videos(
    input_df=cayla_df,
    video_path=video_path,
    pipe_asr=pipe_asr
)


# Then process all prompts (inserting transcriptions only for the 3 specified prompts)
results_df = process_all_prompts(
    input_df=cayla_df_with_transcriptions,
    prompt_dict=prompt_dict,
    video_path=video_path,
    analyze_video=analyze_video,
    nami_df=nami_df,
    natalia_df=natalia_df
)

# Save results
results_df.to_csv("kangaroo_results.csv", index=False)


In [41]:
results_df.head(20)

In [42]:
results_df.shape

In [43]:
os.getcwd()

In [44]:
cayla_path = '/localdisk4/panwla/Projects/park_vlm/Annotations/Cayla'
os.chdir(cayla_path)

cayla_df_with_transcriptions.to_csv("cayla_df_with_transcriptions.csv", index=False)

In [45]:
# === Empty DataFrame to collect results ===
columns = ["Video_file", "Q_ID", "Prompt_name", "Model_Name", "Cayla_Resp", "Nami_Resp", "Natalia_Resp"]
output = pd.DataFrame(columns=columns)

for row_idx, record in enumerate(cayla_df["video"][0:1]):  # loop over first row
    if not isinstance(record, str):
        continue
    # Find matching video file from available .mp4 files
    matched_file = None
    for f in available_videos:
        if record[-25:] in f:
            matched_file = f
            break

    if matched_file:
        full_video_path = os.path.join(video_path, matched_file)
        print(f"\n🎬 full_video_path : {full_video_path}")
        print(f"\n🎬 Processing video: {matched_file}")

        # === Find corresponding rows in other annotator DataFrames based on 'video' column ===
        cayla_row = cayla_df[cayla_df["video"] == record]

        nami_row = nami_df[nami_df["video"] == record]
        natalia_row = natalia_df[natalia_df["video"] == record]

        # Print matched row indices (if found)
        if not nami_row.empty:
            print(f"✅ Matched Nami row index: {nami_row.index[0]}, and video: {nami_row['video']}")
        else:
            print("❌ No match found in nami_df")

        if not natalia_row.empty:
            print(f"✅ Matched Natalia row index: {natalia_row.index[0]}, and video: {natalia_row['video']}")
        else:
            print("❌ No match found in natalia_df")

        prompt_columns = list(cayla_df.columns[0:13])  # convert to list

        for idx, (prompt_name, prompt_text) in enumerate(prompt_dict.items(), start=1):
            cayla_resp = cayla_row[prompt_columns[idx - 1]].values[0] if not cayla_row.empty else ""
            nami_resp = nami_row[prompt_columns[idx - 1]].values[0] if not nami_row.empty else ""
            natalia_resp = natalia_row[prompt_columns[idx - 1]].values[0] if not natalia_row.empty else ""

            response = analyze_video(full_video_path, prompt_text)

            new_row = {
                "Video_file": matched_file,
                "Q_ID": idx,
                "Prompt_name": prompt_name,
                "Model_Name": "Kangaroo",
                "Output": response,
                "Cayla_Resp": cayla_resp,
                "Nami_Resp": nami_resp,
                "Natalia_Resp": natalia_resp
            }

            output = pd.concat([output, pd.DataFrame([new_row])], ignore_index=True) # type: ignore

    else:
        print(f"❌ No matching video found for: {record}")

In [46]:
output['Video_file'].iloc[0]

In [47]:
output.head(15)

In [48]:
output.to_csv("cayla_model_outputs.csv", index=False)


In [49]:
import whisper

# Load the pre-trained Whisper model
model = whisper.load_model("base")  # options: "tiny", "base", "small", "medium", "large"

# Transcribe an audio file
result = model.transcribe(full_video_path)  # or .wav, .m4a, etc.

# Print the transcript
print("Transcription:")
print(result["text"])
